In [1]:
import pandas as pd
from pprint import pprint

ARQUIVO_DADOS = 'dados_oceans14_final.csv'

In [2]:
df_oceans = pd.read_csv(ARQUIVO_DADOS)
df_oceans.columns

Index(['P_L', 'ano', 'margem_liq', 'divida_liq_EBITDA', 'liq_corrente',
       'liq_imediata', 'LPA', 'VPA', 'P_VP', 'P_EBIT', 'P_EBITDA', 'P_ativos',
       'margem_EBITDA', 'PSR', 'ROE', 'ROA', 'ROIC', 'EV_EBIT', 'EV_EBITDA',
       'DY', 'tick'],
      dtype='object')

In [3]:
def somar_posicoes(df_rank, list_indicadores):
    linhas_posi = []
    for tick, linha in df_rank.iterrows():
        soma_posi = 0
        for i in range(1, len(linha), 1):
            soma_posi += linha[i]
        linhas_posi.append([tick, soma_posi])
    
    df_posi = pd.DataFrame(linhas_posi, columns=['tick', 'soma_posi'])
    df_posi.set_index('tick', inplace=True)
    return df_rank.join(df_posi).copy()

In [4]:
def calcular_ranking(df_completo, list_indicadores, ano):
    '''
    DataFrame - dados completos
    list_indicadores - [['TICKER', bool (O primeiro lugar é o menor valor?)], [Ticker, bool], ...]
    '''
    df_rankings = df_completo[df_completo['ano'] == ano][['tick', 'ano']].copy()
    df_rankings.set_index('tick', inplace=True)

    for indicador, asc in list_indicadores:
        df_indicador = df_completo[df_completo['ano'] == ano][['tick', indicador]].copy()
        df_indicador.set_index('tick', inplace=True)

        df_indicador['rank_'+indicador] = df_indicador[indicador].rank(ascending=asc)
        df_indicador.drop(indicador, axis=1, inplace=True)

        df_rankings = df_rankings.join(df_indicador)
    
    df_rankings = somar_posicoes(df_rankings, list_indicadores)
    return df_rankings.sort_values('soma_posi').copy()

In [5]:
df_oceans.columns

Index(['P_L', 'ano', 'margem_liq', 'divida_liq_EBITDA', 'liq_corrente',
       'liq_imediata', 'LPA', 'VPA', 'P_VP', 'P_EBIT', 'P_EBITDA', 'P_ativos',
       'margem_EBITDA', 'PSR', 'ROE', 'ROA', 'ROIC', 'EV_EBIT', 'EV_EBITDA',
       'DY', 'tick'],
      dtype='object')

In [6]:
def analisar_indicadores(lista_indicadores, ano):
    ''' A partir de uma lista de indicadores e um ano, retorna uma dataframe com o ranking de cada empresa
    recebe:
    dict - 'indicador': [min, max]
    int - ano

    retorna
    DataFrame - tabela com: tick, posicao geral, posicao normalizada, posicao no indicador 1, 2, 3, ..., n
    '''
    DICT_ASC = {
        'divida_liq_EBITDA': ['divida_liq_EBITDA', True],
        'ROIC': ['ROIC', False],
        'VPA': ['VPA', True],
        'ROA': ['ROA', False],
        'P_L': ['P_L', True],
        'PSR': ['PSR', False],
        'P_VP': ['P_VP', True],
        'P_EBIT': ['P_EBIT', True],
        'P_EBITDA': ['P_EBITDA', True],
        'DY': ['DY', False],
        'ROE': ['ROE', False],
        'P_ativos': ['P_ativos', True],
        'margem_EBITDA': ['margem_EBITDA', False],
        'LPA': ['LPA', False],
        'margem_liq': ['margem_liq', False],
        'EV_EBITDA': ['EV_EBITDA', True],
        'P_EBITDA': ['P_EBITDA', True],
        'EV_EBIT': ['EV_EBIT', True],
        'liq_corrente': ['liq_corrente', False],
        'liq_imediata': ['liq_imediata', False]
    }

    lista_duplas = []
    for indicador in lista_indicadores:
        lista_duplas.append(DICT_ASC[indicador])
    
    #df_indicadores = pd.read_csv(ARQUIVO_DADOS)
    df_indicadores = filtrar_indicadores(lista_indicadores)
    
    df_final = calcular_ranking(df_indicadores, lista_duplas, ano)

    df_final['posi_normalizada'] = round((100*df_final['soma_posi'])/(len(df_final)*len(indicadores)),2)

    colunas_certas = list(df_final.columns)
    colunas_certas.remove('ano')
    colunas_certas.remove('soma_posi')
    colunas_certas.remove('posi_normalizada')
    colunas_certas = ['soma_posi', 'posi_normalizada'] + colunas_certas

    return df_final[colunas_certas].copy()


In [7]:
def analisar_anos_anteriores(indicadores, tamanho_carteira, ano_inicial):
    ''' Analisar a melhor carteira de tamanho (tamanho_carteira) ao longo dos ultimos anos
    Recebe:
    dict - 'indicador': [min, max]
    int - quantidade de ativos
    int - ano inicial

    Retorna:
    dict - 'ano': [ativo1, ativo2, ..., ativoN]
    '''
    ativos_por_ano = {}

    for ano in range(ano_inicial, 2021, 1):
        df_ano_atual = analisar_indicadores(indicadores, ano)
        df_ano_atual = df_ano_atual.head(10).copy()
        #melhores_empresas = list(df_ano_atual['tick'])
        melhores_empresas = list(df_ano_atual.index)
        ativos_por_ano[str(ano)] = melhores_empresas
    
    for ano in ativos_por_ano:
        ativos_por_ano[ano] = ativos_por_ano[ano] + ([''] * (tamanho_carteira - len(ativos_por_ano[ano])))
    
    return ativos_por_ano


In [8]:
def filtrar_indicadores(dict_indicadores):
    df_indicadores = pd.read_csv(ARQUIVO_DADOS)

    for indicador in dict_indicadores:
        minimo, maximo = dict_indicadores[indicador]
        df_indicadores = df_indicadores[(df_indicadores[indicador] >= minimo) & (df_indicadores[indicador] <= maximo)].copy()

    return df_indicadores.copy()

In [9]:
indicadores = {'ROIC': [0.1, 1]}
#indicadores = {'P_L': [1, 10], 'liq_corrente': [1,2.5]}
ano = 2018

In [10]:
df_filtrado = filtrar_indicadores(indicadores)
df_filtrado

,P_L,ano,margem_liq,divida_liq_EBITDA,liq_corrente,liq_imediata,LPA,VPA,P_VP,P_EBIT,P_EBITDA,P_ativos,margem_EBITDA,PSR,ROE,ROA,ROIC,EV_EBIT,EV_EBITDA,DY,tick
2,27.34,2010,0.08,0.02,2.81,1.19,0.99,7.13,3.79,36.30,24.01,2.75,0.09,2.18,0.14,0.10,0.10,163.95,108.42,0.0144,YDUQ3
4,30.87,2012,0.08,0.39,2.65,0.73,1.35,8.61,4.85,22.99,16.38,2.77,0.15,2.48,0.16,0.09,0.14,69.10,49.21,0.0048,YDUQ3
5,24.34,2013,0.14,0.18,4.38,2.55,0.84,5.17,3.95,23.95,18.97,2.80,0.18,3.46,0.16,0.12,0.18,38.77,30.69,0.0132,YDUQ3
6,17.46,2014,0.18,0.25,3.71,1.80,1.36,7.64,3.12,17.51,14.02,2.12,0.22,3.10,0.18,0.12,0.15,23.55,18.86,0.0084,YDUQ3
7,8.85,2015,0.17,0.39,2.84,0.93,1.58,8.71,1.60,8.32,6.32,0.98,0.23,1.46,0.18,0.11,0.14,20.37,15.47,0.0237,YDUQ3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962,23.38,2003,0.09,0.80,1.53,0.00,0.83,5.26,3.67,11.55,0.00,1.08,0.00,2.10,0.16,0.05,0.10,7.83,0.00,0.0000,EMBR3
1963,8.96,2004,0.12,0.79,1.91,0.00,1.76,6.24,2.53,6.52,0.00,0.80,0.00,1.11,0.28,0.09,0.14,5.69,0.00,0.0000,EMBR3
1972,17.52,2013,0.06,0.60,1.99,0.91,1.08,11.64,1.62,8.60,6.17,0.58,0.16,1.01,0.09,0.03,0.11,3.33,2.39,0.0132,EMBR3
1980,6.17,2008,0.04,0.44,1.98,0.58,0.42,4.80,0.54,1.62,1.35,0.22,0.18,0.24,0.09,0.04,0.19,1.12,0.93,0.0000,PMAM3


In [11]:
df_ano_selecionado = analisar_indicadores(indicadores, ano)
df_ano_selecionado.head(10)

,soma_posi,posi_normalizada,rank_ROIC
tick,,,
WIZS3,1.0,1.27,1.0
BBSE3,2.0,2.53,2.0
SMLS3,3.0,3.80,3.0
ODPV3,4.0,5.06,4.0
MGLU3,5.0,6.33,5.0
UNIP6,6.0,7.59,6.0
CGAS5,7.5,9.49,7.5
ARZZ3,7.5,9.49,7.5
PRIO3,9.0,11.39,9.0


In [12]:
ano_inicial = 2015
tamanho_carteira = 10
melhores_por_ano = analisar_anos_anteriores(indicadores, tamanho_carteira, ano_inicial)
#pprint(melhores_por_ano)
df_melhores_por_ano = pd.DataFrame(melhores_por_ano)
df_melhores_por_ano

,2015,2016,2017,2018,2019,2020
0,BBSE3,PRIO3,ODPV3,WIZS3,WIZS3,WIZS3
1,SMLS3,TRPL4,BBSE3,BBSE3,SMLS3,SMLS3
2,ODPV3,SMLS3,WIZS3,SMLS3,CGAS5,CGAS5
3,ABEV3,BBSE3,MGLU3,ODPV3,ODPV3,ODPV3
4,ARZZ3,ODPV3,ARZZ3,MGLU3,TGMA3,TGMA3
5,NTCO3,CGAS5,BRAP4,UNIP6,QUAL3,QUAL3
6,LAME4,ARZZ3,VULC3,CGAS5,ARZZ3,ARZZ3
7,CGAS5,ABEV3,ABEV3,ARZZ3,CEAB3,VIVA3
8,EGIE3,MGLU3,GRND3,PRIO3,VIVA3,BEEF3
9,LREN3,TIET11,PARD3,YDUQ3,ABEV3,ABEV3
